### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [ ]:
# Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path

In [ ]:
# Defining paths
GROUP_DATA_DIR = Path("/Users/harshul/projects/kgwebsite/group-data")

MEMBERS_DIR_PATH = GROUP_DATA_DIR / "members/"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / "content"
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "templates"
HOSTING_PATH = GROUP_DATA_DIR.parent / "kerzendorf-group.github.io"

In [ ]:
# Function for creating proper html file names
def page_link(a):
    return a.replace(" ", "_") if " " in a else a


In [ ]:
# Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH), extensions=["jinja2.ext.loopcontrols"]
)
environment.globals["page_link"] = page_link

In [ ]:
content_id_data = {"article_id": [], "category": [], "date": [], "tags": []}


for json_file in os.listdir(CONTENT_DIR_PATH):
    if json_file.endswith(".json"):
        json_path = os.path.join(CONTENT_DIR_PATH, json_file)
        with open(json_path, "r") as file:
            info = json.load(file)
            if info.get("display"):
                content_id_data["article_id"].append(info.get("article_id"))
                content_id_data["category"].append(info.get("category"))
                content_id_data["date"].append(info.get("article_date"))
                content_id_data["tags"].append(info.get("tags"))

content_df = pd.DataFrame(content_id_data)
content_df["date"] = pd.to_datetime(content_df["date"], format="%m-%d-%Y")
content_df = (
    content_df.groupby("category")
    .apply(lambda x: x.sort_values("date", ascending=False))
    .reset_index(drop=True)
)

In [ ]:
research_content_unsorted = content_df[
    content_df["tags"].apply(lambda x: any("research" in tag for tag in x))
]
research_content = (
    research_content_unsorted.groupby("category")
    .apply(lambda x: x.sort_values("date", ascending=False))
    .reset_index(drop=True)
)

In [ ]:
news_content_unsorted = content_df[
    content_df["tags"].apply(lambda x: any("news" in tag for tag in x))
]
news_content = news_content_unsorted.sort_values(by="date", ascending=False)

In [ ]:
latest_content_df = pd.DataFrame()

for category in content_df.category.unique():
    latest_data = pd.Series(content_df[content_df.category == category].iloc[0])
    latest_content_df = latest_content_df._append(latest_data, ignore_index=True)

latest_content_df["date"] = pd.to_datetime(latest_content_df["date"], format="%m-%d-%Y")
latest_content_df = latest_content_df.sort_values(by="date", ascending=False)

In [ ]:
json_files = ["general", "homepage", "research", "support", "contact"]
data = {}

for json_file in json_files:
    json_file_path = WEBSITE_DATA_PATH / f"{json_file}.json"

    try:
        with open(json_file_path, "r") as json_var:
            data[json_file] = json.load(json_var)
    except (FileNotFoundError, json.JSONDecodeError):
        pass

##### Homepage

In [ ]:
homepage_template = environment.get_template("homepage.html.j2")

In [ ]:
homepage_content = homepage_template.render(
    general=data["general"],
    homepage=data["homepage"],
    recent_content=latest_content_df.to_dict(orient="records"),
)

In [ ]:
homepage_file_path = HOSTING_PATH / "Index.html"

with open(homepage_file_path, mode="w", encoding="utf-8") as homepage:
    homepage.write(homepage_content)

##### People Page

In [ ]:
def parse_member_data(member_dir):
    member_json_dir = member_dir / "jsons"
    education_experience_list = []
    file_names = ["experiences.json", "education.json"]

    for file_name in file_names:
        file_path = member_json_dir / file_name
        if file_path.exists():
            with open(file_path, "r") as f:
                education_experience_list += json.load(f)
        else:
            print(file_path)

    # Load social links directly
    social_links = {}
    social_links_file_path = member_json_dir / "social_links.json"
    if social_links_file_path.exists():
        with open(social_links_file_path, "r") as f:
            social_links = json.load(f)

    # Load topmost project title
    current_project_title = None
    projects_file_path = member_json_dir / "projects.json"
    if projects_file_path.exists():
        with open(projects_file_path, "r") as f:
            projects = json.load(f)
            if projects:  # Ensure it's not an empty list
                current_project_title = projects[0].get("project_title")

    return pd.DataFrame(education_experience_list), social_links, current_project_title

In [ ]:
# member_dir = Path("/Users/harshul/projects/kgwebsite/group-data/members/wolfgang_kerzendorf")
# parse_member_data(member_dir)

In [ ]:
# Assigning roles


def extract_member_academic_role(education_experience_df):
    # check if columns exist in dataframe
    for column in ["end_date", "group", "institution"]:
        if column not in education_experience_df.columns:
            education_experience_df[column] = None

    current_academic_role = ""

    role_map = {
        "Assistant Professor": "Professor",
        "Professor": "Professor",
        "Visualization Consultant": "Visualization Consultant",
        "Research Consultant": "Research Consultant",
        "Research Software Engineer": "Research Software Engineer",
        "Professorial Assistant": "Professorial Assistant",
        "Visiting Researcher": "Postdoctoral Researcher",
        "Postdoctoral Researcher": "Postdoctoral Researcher",
    }

    degree_map = {
        "Masters": "Graduate Student",
        "PhD": "Postdoctorate",
        "Bachelors": "Graduate Student",
    }

    for _, row in education_experience_df.iterrows():
        role = row.get("role", None)
        degree = row.get("degree", None)

        if not current_academic_role:
            current_academic_role = role_map.get(role, "")

            if degree == "PhD" and pd.isna(row["end_date"]):
                current_academic_role = "Graduate Student"
            elif degree == "Bachelors" and pd.isna(row["end_date"]):
                current_academic_role = "Undergraduate Student"
            elif not current_academic_role and degree in degree_map:
                current_academic_role = degree_map[degree]

    # Check for end dates outside the loop
    has_end_date = all(
        not pd.isna(date) for date in education_experience_df["end_date"]
    )
    is_current_member = not has_end_date

    return current_academic_role, is_current_member

In [ ]:
# for creating two lists

current_people_page_list = []
alumni_people_page_list = []


for member_dir in MEMBERS_DIR_PATH.glob("*"):
    print(member_dir)
    if not (member_info_fname := member_dir / "info.json").exists():
        continue
    else:
        member_info = json.load(open(member_info_fname, "r"))
    education_experience_df, social_links, current_project_title = parse_member_data(
        member_dir
    )
    current_academic_role, is_current_member = extract_member_academic_role(
        education_experience_df
    )

    first_name = member_info["first_name"]
    # print(first_name)
    last_name = member_info["last_name"]
    nickname = member_info.get("nick_name", None)
    id = member_info["id"]
    image_path = member_info["image_path"]
    cover_image_path = member_info["cover_image_path"]

    name = f"{nickname if nickname else first_name} {last_name}"

    member_data = {
        "name": name,
        "academic_role": current_academic_role,
        "id": id,
        "current_project_title": current_project_title,
        "image_path": image_path,
        "cover_image_path": cover_image_path,
    }

    member_data.update(social_links)

    if is_current_member:
        current_people_page_list.append(member_data)
    else:
        alumni_people_page_list.append(member_data)

In [ ]:
# current_people_page_list
# alumni_people_page_list

In [ ]:
people_template = environment.get_template("people.html.j2")

In [ ]:
people_content = people_template.render(
    general=data["general"],
    current_members=current_people_page_list,
    alumni_members=alumni_people_page_list,
)

In [ ]:
people_file_path = HOSTING_PATH / "People.html"

with open(people_file_path, mode="w", encoding="utf-8") as people:
    people.write(people_content)